In [11]:
import numpy as np
import matplotlib.pyplot as plt
import csv

# FIT01 for Fuel Cycle Simulation Reactor Depletion
### This Functionality Isolation Test (FIT) is to test MOX fuel fabrication and depletion functionalities in a reactor module given streams of varying plutonium quality (different isotopics). The four streams are:
### `ref, source_1, source_2, and source_3`.

### ORION is used for this analysis, where a MOX LWR fabrication and reactor module is given four streams with varying plutonium stream. 

### The Fuel Loading Module (FLM) is the `Fuel Fabrication` facility in ORION, that uses cross sections and nu-bars to calculate how to mix the fissile and fertile stream. The datasets with `_ff` are ones using fixed fractions, which means that the FLM is not utilized.

### Depletion is done using cross section data.

### The fissile stream is plutonium and americium in different qualities.
### The fertile stream is depleted uranium (0.238% U235).

In [12]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [13]:
def get_comp_dict(filename):
    with open(filename, 'r') as f:
        lines = f.readlines()
        isotopes = lines[0]
        isotopes = isotopes.split(',')[1:-1]
        isotopes = [x.split('+')[1] for x in isotopes]
        masses = lines[-1]
        masses = masses.split(',')[1:-1]
        masses = np.array([float(x) for x in masses])
        total_mass = sum(masses)
    
    comp_dict = {}
    for indx, val in enumerate(isotopes):
        comp_dict[val] = masses[indx]
    
    # get rid of oxygen
    comp_dict = {key: value for key, value in comp_dict.items()
                 if 'O' not in key}
    
    # from mass to percentage
    comp_dict = {key: value / sum(comp_dict.values()) * 100 for key, value
                 in comp_dict.items()}

    return comp_dict

In [14]:
def show_fracs(comp_dict, search_list=[]):
    filtered_dict = {}
    for key, val in comp_dict.items():
        if val < 1e-10:
            continue
        if len(search_list) > 1:
            for keyword in search_list:
                if keyword.upper() in key:
                    filtered_dict[key] = val
        else:
            filtered_dict[key] = val
    return filtered_dict

In [15]:
def get_am_pu(filtered_dict):
    am_pu = 0
    for key, val in filtered_dict.items():
        if 'AM' in key or 'PU' in key:
            am_pu += val
    return am_pu

In [16]:
def main(file_list):
    data = {}
    for file in file_list:
        comp_dict = get_comp_dict(file)
#        frac_dict = show_fracs(comp_dict, ['U', 'NP', 'PU', 'AM'])
        frac_dict = show_fracs(comp_dict)
        am_pu = get_am_pu(frac_dict)
        filename = file.split('/')[-1]
        filename = filename.split('.csv')[0]
        data.update({filename:{'fraction': frac_dict, 'AM+PU': am_pu}})
    return data

In [17]:
from os import listdir
from os.path import isfile, join
file_list = [f for f in listdir('./results') if isfile(join('./results', f))]
file_list =['./results/' + x for x in file_list]

In [18]:
data = main(file_list)

In [19]:
# show all 
def show_data(data, must_word=''):
    for key, val in data.items():
        if must_word not in key:
            continue
        print('=========================')
        print(key)
        for key2, val2 in val.items():
            if key2 == 'fraction':
                for key3, val3 in val2.items():
                    print('%s = %f' %(key3, val3))
            else:
                print('%s = %f' %(key2, val2))
        print('=========================')
        print('\n')


In [20]:
# show all -> show_data(data)
# show discharge only -> show_data(data, '_discharge')
# show charge only -> show_data(data, '_charge')
# show fixed fraction only -> show_data(data, '_ff')
show_data(data, 'thousand')

thousand
TH230 = 0.000000
TH234 = 0.000000
PA231 = 0.000000
U234 = 0.002799
U235 = 0.221671
U235(1) = 0.000000
U236 = 0.000437
U237 = 0.000000
U238 = 92.779400
NP237 = 0.000224
PU238 = 0.135675
PU239 = 4.357197
PU240 = 1.574561
PU241 = 0.491753
PU242 = 0.350001
AM241 = 0.086283
AM+PU = 6.995470




## major errors spotted:
1. The charge fuel composition does not change
    * the Pu composition changes, obviously becasue the input stream is different.
    * but the U composition does not, which means that the fuel fabrication is mixing the fuel at the same ratio
        * Maybe it's a cross section thing, so I tried using CASMO mox cross section file, still the same.
2. The depleted composition is different from excel
    * U238 drops down to ~80 percent
        * tried power density 10.13 W/g
        * 104.5 W/cc
    * very minor differences across diff pu streams
    * the AM+PU is higher than excel